# Dependencies

In [10]:
from dotenv import load_dotenv
import traceback
from langchain_openai import ChatOpenAI
from agentic_chunker import AgenticChunker
from langchain_community.document_loaders import PyMuPDFLoader
import os

load_dotenv()

True

# Tools inializations

In [30]:
openai_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model="gpt-4o", temperature=0)
chunker = AgenticChunker(api_key = openai_key , model= llm)

In [31]:
file_path = "/home/youssef/github/Modular_RAG/PDFs/1H2025_Earnings_Release.pdf"
loader = PyMuPDFLoader(
    file_path , 
    mode="page"
)

doc = loader.load()

# Chunking technique

In [32]:
# Assuming the chunker processes the entire list of Document objects at once.

print("\n-- Step 1: Processing Document with Agentic Chunker --\n")

# Pass the entire list of Document objects (doc) to a method like 'process_documents'
# You may need to change 'process_documents' to 'run' or 'process' if this fails.
try:
    chunker.process_documents(doc)
    print("Document processing initiated using 'process_documents'.")
except AttributeError:
    # If the method is called 'process'
    try:
        chunker.process(doc)
        print("Document processing initiated using 'process'.")
    except AttributeError:
        # If the method is called 'run'
        try:
            chunker.run(doc)
            print("Document processing initiated using 'run'.")
        except AttributeError:
            print("ERROR: None of the common processing methods were found. The method name is highly specific to your library version.")
            
print("\n-- Step 2: Agentic Chunking Complete. Retrieving Results --\n")

# Retrieve the results using the common retrieval patterns
if hasattr(chunker, 'chunks'):
    hierarchal_chunks = chunker.chunks
    print(f"Successfully generated {len(hierarchal_chunks)} chunks.")
    
    if hierarchal_chunks:
        print("\n--- Example of the First Chunk ---")
        first_chunk_key = next(iter(hierarchal_chunks))
        print(hierarchal_chunks[first_chunk_key])
else:
    try:
        # Try a common retrieval method
        hierarchal_chunks = chunker.get_chunks()
        print(f"Successfully retrieved {len(hierarchal_chunks)} chunks using get_chunks().")
    except AttributeError:
        print("Could not find a standard result retrieval method ('chunks' or 'get_chunks').")


-- Step 1: Processing Document with Agentic Chunker --

ERROR: None of the common processing methods were found. The method name is highly specific to your library version.

-- Step 2: Agentic Chunking Complete. Retrieving Results --

Could not find a standard result retrieval method ('chunks' or 'get_chunks').


In [27]:
print("\n-- Step 1: Initiating Agentic Chunking --\n")

# The correct method is 'agentic_chunking', which takes the list of documents (doc) 
# and returns the new list of hierarchically chunked documents.
try:
    # The 'agentic_chunking' method runs the LLM-powered process
    hierarchal_chunks = chunker.agentic_chunking(doc)
    
    print(f"Successfully generated {len(hierarchal_chunks)} chunks.")

    if hierarchal_chunks:
        print("\n--- Example of the First Chunk ---")
        # Print the page_content and metadata of the first chunk
        print(f"Content:\n{hierarchal_chunks[0].page_content[:500]}...")
        print(f"\nMetadata: {hierarchal_chunks[0].metadata}")
        
except Exception as e:
    print(f"An error occurred during chunking: {e}")
    print("Please ensure your environment variables (like OpenAI API Key) are correctly set, as this is an LLM-powered process.")


-- Step 1: Initiating Agentic Chunking --

An error occurred during chunking: AgenticChunker.agentic_chunking() missing 1 required positional argument: 'seperator'
Please ensure your environment variables (like OpenAI API Key) are correctly set, as this is an LLM-powered process.
